# Data Preparation

In [2]:
# Import package
import pandas as pd

In [3]:
# Import data
buku = pd.read_csv("https://github.com/zygmuntz/goodbooks-10k/blob/master/books.csv?raw=true")

# EDA & Completing Missing Values

In [188]:
books.columns

Index(['book_id', 'goodreads_book_id', 'best_book_id', 'work_id',
       'books_count', 'isbn', 'isbn13', 'authors', 'original_publication_year',
       'original_title', 'title', 'language_code', 'average_rating',
       'ratings_count', 'work_ratings_count', 'work_text_reviews_count',
       'ratings_1', 'ratings_2', 'ratings_3', 'ratings_4', 'ratings_5',
       'image_url', 'small_image_url'],
      dtype='object')

In [10]:
buku.head()

,book_id,goodreads_book_id,best_book_id,work_id,books_count,isbn,isbn13,authors,original_publication_year,original_title,...,ratings_count,work_ratings_count,work_text_reviews_count,ratings_1,ratings_2,ratings_3,ratings_4,ratings_5,image_url,small_image_url
0,1,2767052,2767052,2792775,272,439023483,9.780439e+12,Suzanne Collins,2008.0,The Hunger Games,...,4780653,4942365,155254,66715,127936,560092,1481305,2706317,https://images.gr-assets.com/books/1447303603m...,https://images.gr-assets.com/books/1447303603s...
1,2,3,3,4640799,491,439554934,9.780440e+12,"J.K. Rowling, Mary GrandPré",1997.0,Harry Potter and the Philosopher's Stone,...,4602479,4800065,75867,75504,101676,455024,1156318,3011543,https://images.gr-assets.com/books/1474154022m...,https://images.gr-assets.com/books/1474154022s...
2,3,41865,41865,3212258,226,316015849,9.780316e+12,Stephenie Meyer,2005.0,Twilight,...,3866839,3916824,95009,456191,436802,793319,875073,1355439,https://images.gr-assets.com/books/1361039443m...,https://images.gr-assets.com/books/1361039443s...
3,4,2657,2657,3275794,487,61120081,9.780061e+12,Harper Lee,1960.0,To Kill a Mockingbird,...,3198671,3340896,72586,60427,117415,446835,1001952,1714267,https://images.gr-assets.com/books/1361975680m...,https://images.gr-assets.com/books/1361975680s...
4,5,4671,4671,245494,1356,743273567,9.780743e+12,F. Scott Fitzgerald,1925.0,The Great Gatsby,...,2683664,2773745,51992,86236,197621,606158,936012,947718,https://images.gr-assets.com/books/1490528560m...,https://images.gr-assets.com/books/1490528560s...


Cek apakah ada empty/missing value

In [ ]:
buku.isna().sum()

book_id                         0
goodreads_book_id               0
best_book_id                    0
work_id                         0
books_count                     0
isbn                          700
isbn13                        585
authors                         0
original_publication_year      21
original_title                585
title                           0
language_code                1084
average_rating                  0
ratings_count                   0
work_ratings_count              0
work_text_reviews_count         0
ratings_1                       0
ratings_2                       0
ratings_3                       0
ratings_4                       0
ratings_5                       0
image_url                       0
small_image_url                 0
dtype: int64

## Melengkapi original_publication_year

In [ ]:
missing_pubyear_id = buku[buku['original_publication_year'].isna()]['goodreads_book_id'].tolist()
missing_pubyear_id

[3609760,
 90072,
 856917,
 36,
 8724779,
 349254,
 9832370,
 5171894,
 17733861,
 1044141,
 9748274,
 48811,
 9528186,
 809849,
 765148,
 197436,
 351023,
 11318,
 6054190,
 1056627,
 6614886]

In [ ]:
import requests
from bs4 import BeautifulSoup
import re

Ambil aja dari Goodreads, tapi enggak 100% yakin itu *original* publication year-nya. Gapapa lah.

In [ ]:
def get_pub_year(i):
    url = f"https://www.goodreads.com/book/show/{i}"
    html_page = requests.get(url)
    soup = BeautifulSoup(html_page.content, 'html.parser')
    try:
        pub_text = soup.select('div.row')[1].text.strip()
        pub_year = int(re.search(r'\d{4}', pub_text).group(0))
    except:
        pub_year = ''

    return pub_year

In [ ]:
pub_year = []
for id in missing_pubyear_id:
  pub_year.append(get_pub_year(id))
pub_year

[2008,
 1998,
 2006,
 2003,
 2009,
 2003,
 2010,
 2009,
 2013,
 2004,
 2010,
 2006,
 2010,
 2007,
 2001,
 2000,
 2004,
 1988,
 2009,
 2007,
 2009]

In [ ]:
buku.loc[buku['original_publication_year'].isna(), 'original_publication_year'] = pub_year
buku['original_publication_year'].isna().any()

False

## Melengkapi original title (dan isbn13?)

In [ ]:
buku.isna().sum()

book_id                         0
goodreads_book_id               0
best_book_id                    0
work_id                         0
books_count                     0
isbn                          700
isbn13                        585
authors                         0
original_publication_year       0
original_title                585
title                           0
language_code                1084
average_rating                  0
ratings_count                   0
work_ratings_count              0
work_text_reviews_count         0
ratings_1                       0
ratings_2                       0
ratings_3                       0
ratings_4                       0
ratings_5                       0
image_url                       0
small_image_url                 0
dtype: int64

Jumlah missing values `original_title` dan `isbn` sama, jadi mungkin berasal dari rows yang sama pula.

In [ ]:
missing_ori_title = buku.loc[buku['original_title'].isna(), :].goodreads_book_id
missing_ori_title

74        227443
142     18143977
209       345627
214      9969571
256        13023
          ...   
9942      653559
9944    18943009
9956     6332526
9957     7494887
9970    12602078
Name: goodreads_book_id, Length: 585, dtype: int64

In [ ]:
missing_isbn13 = buku.loc[buku['isbn13'].isna(), :].goodreads_book_id
missing_isbn13

105      9418327
120         7604
161        49552
187     10664113
202     11505797
          ...   
9928     6614886
9944    18943009
9958    13178410
9966     5295735
9970    12602078
Name: goodreads_book_id, Length: 585, dtype: int64

Ouh, ternyata lumayan beda. Oke, selesaikan terpisah.

In [ ]:
missing_ori_title = missing_ori_title.to_list()

In [ ]:
def get_ori_title(i):
  url = f"https://www.goodreads.com/book/show/{i}"
  headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/50.0.2661.102 Safari/537.36'}
  html_page = requests.Session().get(url, headers=headers)
  soup = BeautifulSoup(html_page.content, 'html.parser')
  try:
      ori_title = soup.select('.infoBoxRowItem')[0].text.strip()
  except:
      ori_title = None
      
  return ori_title

585 kebanyakan? Saatnya threading!

In [ ]:
import concurrent.futures
import time
from tqdm import tqdm

In [ ]:
while buku['original_title'].isna().sum() > 0:
  missing_ori_title = buku.loc[buku['original_title'].isna(), :].goodreads_book_id.to_list()
  found_ori_title = ['' for x in missing_ori_title]

  with concurrent.futures.ThreadPoolExecutor() as executor:
    # with concurrent.futures.ThreadPoolExecutor() as executor:
    #   future = executor.map(get_songs, alphabets)
    #   songs_kapanlagi = [song for f in future for song in f]

    future = executor.map(get_ori_title, missing_ori_title)
    for i,f in enumerate(tqdm(future, total=len(missing_ori_title))):
      found_ori_title[i] = f
      time.sleep(1e-3)

  buku.loc[buku['original_title'].isna(), 'original_title'] = found_ori_title

  print(len(found_ori_title))
  print(buku['original_title'].isna().sum())

  0%|          | 0/517 [00:00<?, ?it/s]

585
517


  0%|          | 0/496 [00:00<?, ?it/s]

517
496


  0%|          | 0/471 [00:00<?, ?it/s]

...

1
1


  0%|          | 0/1 [00:00<?, ?it/s]

1
1


100%|██████████| 1/1 [00:05<00:00,  5.28s/it]

1
0


In [ ]:
buku.to_csv('buku_v2.csv', index=False)

from google.colab import files
files.download( 'buku_v2.csv' )

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
missing_isbn13 = missing_isbn13.to_list()

In [ ]:
missing_isbn13[:5]

[9418327, 7604, 49552, 10664113, 11505797]

In [ ]:
def get_isbn13(i):
  url = f"https://www.goodreads.com/book/show/{i}"
  headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/50.0.2661.102 Safari/537.36'}
  html_page = requests.Session().get(url, headers=headers)
  soup = BeautifulSoup(html_page.content, 'html.parser')
  try:
      isbn13 = soup.select('span', itemprop='isbn')
  except:
      isbn13 = None
      
  return isbn13

get_isbn13(missing_isbn13[2])

In [ ]:
while buku['isbn13'].isna().sum() > 0:
  missing_ori_title = buku.loc[buku['isbn13'].isna(), :].goodreads_book_id.to_list()
  found_ori_title = ['' for x in missing_ori_title]

  with concurrent.futures.ThreadPoolExecutor() as executor:
    # with concurrent.futures.ThreadPoolExecutor() as executor:
    #   future = executor.map(get_songs, alphabets)
    #   songs_kapanlagi = [song for f in future for song in f]

    future = executor.map(get_ori_title, missing_ori_title)
    for i,f in enumerate(tqdm(future, total=len(missing_ori_title))):
      found_ori_title[i] = f
      time.sleep(1e-3)

  buku.loc[buku['original_title'].isna(), 'original_title'] = found_ori_title

  print(len(found_ori_title))
  print(buku['original_title'].isna().sum())

Ternyata emang enggak semua buku ada isbn13-nya :/

Oke, bisa diabaikan.

# Futher Pre-Processing

In [ ]:
import pandas as pd

link = 'https://github.com/ilmaaliyaf/macapedia_recommender/blob/main/dataset/buku_v2.csv?raw=true'
buku = pd.read_csv(link)
buku.isna().sum()

book_id                         0
goodreads_book_id               0
best_book_id                    0
work_id                         0
books_count                     0
isbn                          700
isbn13                        585
authors                         0
original_publication_year       0
original_title                  0
title                           0
language_code                1084
average_rating                  0
ratings_count                   0
work_ratings_count              0
work_text_reviews_count         0
ratings_1                       0
ratings_2                       0
ratings_3                       0
ratings_4                       0
ratings_5                       0
image_url                       0
small_image_url                 0
dtype: int64

## Inspecting Goodreads

> Purpose: get `goodreads_id` of top books for each major genres in few years back

Strategy:
1. Get `.xml` file for each book page
2. Process the `.xml` offline/online and avoid making any `request` to the website, get the book metadata, ratings, etc.

In [ ]:
# selecting year
years = [x for x in range(2017, 2021)]

# selecting big genres
genres = ['fiction-books',
          'mystery-thriller-books',
          'historical-fiction-books',
          'fantasy-books',
          'romance-books',
          'science-fiction-books',
          'horror-books',
          'humor-books',
          'nonfiction-books',
          'memoir-autobiography-books',
          'history-biography-books',
          'science-technology-books',
          'food-cookbooks',
          'graphic-novels-comics',
          'poetry-books',
          'young-adult-fiction-books',
          'young-adult-fantasy-books',
          'childrens-books',
          'picture-books']

urls = [f'https://www.goodreads.com/choiceawards/best-{genre}-{year}' 
        for year in years for genre in genres]

year, genres

([2017, 2018, 2019, 2020],
 ['fiction-books',
  'mystery-thriller-books',
  'historical-fiction-books',
  'fantasy-books',
  'romance-books',
  'science-fiction-books',
  'horror-books',
  'humor-books',
  'nonfiction-books',
  'memoir-autobiography-books',
  'history-biography-books',
  'science-technology-books',
  'food-cookbooks',
  'graphic-novels-comics',
  'poetry-books',
  'young-adult-fiction-books',
  'young-adult-fantasy-books',
  'childrens-books',
  'picture-books'])

In [ ]:
import requests
from bs4 import BeautifulSoup
import re

In [ ]:
url = urls[0]
page = requests.get(url)
soup = BeautifulSoup(page.content, 'lxml')
# soup

In [ ]:
_ = soup.select('.js-tooltipTrigger')
book_id = [x.get('data-resource-id') for x in _]
len(urls) * len(book_id)

1600

In [ ]:
len([])

0

In [ ]:
# wrap it out
book_id = []
for url in urls:
  _ = []
  while len(_) == 0:
    page = requests.get(url)
    soup = BeautifulSoup(page.content, 'lxml')
    _ = [x.get('data-resource-id') for x in soup.select('.js-tooltipTrigger')]
    if len(_) == 0:
      time.sleep(1)
  book_id += _
  print(f'from {url} added {len(_)} ids to the list')

print('total ids:', len(book_id))

from https://www.goodreads.com/choiceawards/best-fiction-books-2017 added 20 ids to the list
from https://www.goodreads.com/choiceawards/best-mystery-thriller-books-2017 added 20 ids to the list
from https://www.goodreads.com/choiceawards/best-historical-fiction-books-2017 added 20 ids to the list
from https://www.goodreads.com/choiceawards/best-fantasy-books-2017 added 20 ids to the list
from https://www.goodreads.com/choiceawards/best-romance-books-2017 added 20 ids to the list
from https://www.goodreads.com/choiceawards/best-science-fiction-books-2017 added 20 ids to the list
from https://www.goodreads.com/choiceawards/best-horror-books-2017 added 20 ids to the list
from https://www.goodreads.com/choiceawards/best-humor-books-2017 added 20 ids to the list
from https://www.goodreads.com/choiceawards/best-nonfiction-books-2017 added 20 ids to the list
from https://www.goodreads.com/choiceawards/best-memoir-autobiography-books-2017 added 20 ids to the list
from https://www.goodreads.co

In [ ]:
import pickle
# with open('book_id.pkl', 'wb') as handle:
#     pickle.dump(book_id, handle)

with open('book_id.pkl', 'rb') as handle:
  book_id = pickle.load(handle)
len(book_id)

1520

## Getting lxml files

In [ ]:
import requests
from bs4 import BeautifulSoup
import time
# import re

def get_xml_file(id):
  url = f'https://www.goodreads.com/book/show/{id}'
  page = requests.get(url)
  # id = re.search(r'\d+', url).group(0)
  while page.status_code != 200:
    page = requests.get(url)
    time.sleep(0.1)
  with open(id + '.xml', 'wb') as f:
    f.write(page.content)
  return id

In [ ]:
# test
id = '36100966'
get_xml_file(id)

'36100966'

In [ ]:
import concurrent.futures
from tqdm import tqdm

In [ ]:
import concurrent.futures
from tqdm import tqdm

with concurrent.futures.ThreadPoolExecutor() as executor:
  future = executor.map(get_xml_file, book_id)
  for i,f in enumerate(tqdm(future, total=len(book_id))):
    print(f' {i} sucessfully saved {id}.xml')
    time.sleep(1e-5)

  0%|          | 1/1520 [00:02<1:14:27,  2.94s/it]

 0 sucessfully saved 36100966.xml


  0%|          | 6/1520 [00:03<37:43,  1.50s/it]

 1 sucessfully saved 36100966.xml
 2 sucessfully saved 36100966.xml
 3 sucessfully saved 36100966.xml
 4 sucessfully saved 36100966.xml
 5 sucessfully saved 36100966.xml
 6 sucessfully saved 36100966.xml
 7 sucessfully saved 36100966.xml
 8 sucessfully saved 36100966.xml
 9 sucessfully saved 36100966.xml


  1%|          | 11/1520 [00:05<29:42,  1.18s/it]

...

 1481 sucessfully saved 36100966.xml
 1482 sucessfully saved 36100966.xml
 1483 sucessfully saved 36100966.xml
 1484 sucessfully saved 36100966.xml
 1485 sucessfully saved 36100966.xml
 1486 sucessfully saved 36100966.xml
 1487 sucessfully saved 36100966.xml
 1488 sucessfully saved 36100966.xml
 1489 sucessfully saved 36100966.xml


 98%|█████████▊| 1491/1520 [27:36<01:35,  3.30s/it]

 1490 sucessfully saved 36100966.xml
 1491 sucessfully saved 36100966.xml
 1492 sucessfully saved 36100966.xml
 1493 sucessfully saved 36100966.xml
 1494 sucessfully saved 36100966.xml
 1495 sucessfully saved 36100966.xml
 1496 sucessfully saved 36100966.xml
 1497 sucessfully saved 36100966.xml
 1498 sucessfully saved 36100966.xml
 1499 sucessfully saved 36100966.xml
 1500 sucessfully saved 36100966.xml
 1501 sucessfully saved 36100966.xml
 1502 sucessfully saved 36100966.xml
 1503 sucessfully saved 36100966.xml
 1504 sucessfully saved 36100966.xml
 1505 sucessfully saved 36100966.xml
 1506 sucessfully saved 36100966.xml
 1507 sucessfully saved 36100966.xml
 1508 sucessfully saved 36100966.xml
 1509 sucessfully saved 36100966.xml
 1510 sucessfully saved 36100966.xml
 1511 sucessfully saved 36100966.xml
 1512 sucessfully saved 36100966.xml
 1513 sucessfully saved 36100966.xml
 1514 sucessfully saved 36100966.xml
 1515 sucessfully saved 36100966.xml
 1516 sucessfully saved 36100966.xml
 

100%|██████████| 1520/1520 [27:46<00:00,  1.10s/it]

 1518 sucessfully saved 36100966.xml
 1519 sucessfully saved 36100966.xml


In [ ]:
! zip -r xml.zip *

  adding: 11503920.xml (deflated 84%)
  adding: 17333180.xml (deflated 83%)
  adding: 17699853.xml (deflated 83%)
  adding: 20670612.xml (deflated 85%)
  adding: 21801668.xml (deflated 84%)
  adding: 22062213.xml (deflated 84%)
  adding: 22303684.xml (deflated 84%)
  adding: 22318476.xml (deflated 83%)
  adding: 22552026.xml (deflated 85%)
  adding: 23106013.xml (deflated 84%)
  adding: 23197837.xml (deflated 83%)
  adding: 23267873.xml (deflated 84%)
  adding: 23308087.xml (deflated 82%)
  adding: 23367261.xml (deflated 84%)
  adding: 23401438.xml (deflated 83%)
  adding: 23447923.xml (deflated 83%)
  adding: 23766634.xml (deflated 80%)
  adding: 24154742.xml (deflated 86%)
  adding: 24212581.xml (deflated 82%)
  adding: 24612045.xml (deflated 84%)
  adding: 24763621.xml (deflated 82%)
  adding: 24909347.xml (deflated 83%)
  adding: 25446343.xml (deflated 83%)
  adding: 25489134.xml (deflated 82%)
  adding: 25613472.xml (deflated 84%)
  adding: 25740412.xml (deflated 81%)
  adding: 25

In [17]:
!cp xml.zip /content/drive/MyDrive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Inspecting the web page (xml) 

In [134]:
import pickle

with open('newer-books/book_id.pkl', 'rb') as handle:
    book_ids = pickle.load(handle)
len(book_ids)

1520

In [176]:
import re
from bs4 import BeautifulSoup
import requests

In [193]:
# book_id,goodreads_book_id,best_book_id,work_id,books_count,isbn,isbn13,authors,original_publication_year,original_title,title,language_code,average_rating,ratings_count,work_ratings_count,work_text_reviews_count,ratings_1,ratings_2,ratings_3,ratings_4,ratings_5,image_url,small_image_url

# i += 1
# with open(f'newer-books/{book_ids[i]}.xml', 'rb') as page:
#     soup = BeautifulSoup(page, 'html.parser')

# book_id = int
# goodreads_book_id = int
# books_count = int(''.join(d for d in soup.select('.row')[0].text.strip() 
#                 if d.isdigit()))
# isbn = soup.select('.infoBoxRowItem')[1].text.strip()
# try: isbn13 = int(isbn[isbn.find(':')+2:-1])
# except: isbn13 = None
# try: isbn = int(isbn[:isbn.find('\n')])
# except: isbn = None
# authors = soup.select('.authorName')[0].text.strip()
# original_publication_year = int(''.join(i for i 
#                               in soup.select('.row')[1].text.strip() 
#                               if i.isdigit())[-4:])
# original_title = soup.select('.infoBoxRowItem')[0].text.strip()
# title = soup.select('#bookTitle')[0].text.strip()
# language_code = soup.select('.infoBoxRowItem')[2].text.strip()
# average_rating = float(soup.find_all('span', {'itemprop': 'ratingValue'})[0].text.strip())
# rating_count = int(''.join(filter(str.isdigit, 
#                     soup.find_all('meta', {'itemprop': 'ratingCount'})[0].text.strip())))
# image_url = soup.find_all('img', {'id': 'coverImage'})[0].get('src')
# try: 
#     t = re.search(r'_(.+)_', image_url).span()
#     small_image_url = image_url[:t[0]] + '_SX98_' + image_url[t[1]:]
# except:
#     small_image_url = image_url[:-3] + '_SX98_' + image_url[-4:]
# try: requests.get(small_image_url).status_code == 200
# except: small_image_url = image_url

# cdata = soup.find_all(text=re.compile("CDATA"))
# [r_5, r_4, r_3, r_2, r_1] = [int(x) for x in re.findall(r'% \((\d+)\)', cdata[7])]

tags = [int(_) for _ in eval(re.search(r'tags", (.+)\)', cdata[2]).group(1))]
tags_count = [int(''.join(i for i in g.text.strip() if i.isdigit()))
                for g in soup.find_all('div', {'class': 'right'})]

# {'book_id': int, 'goodreads_book_id': int,
#  'books_count': books_count, 'isbn': isbn, 'isbn13': isbn13,
#  'authors': authors, 'original_publication_year': original_publication_year,
#  'original_title': original_title, 'title': title, 'language_code': language_code,
#  'average_rating': average_rating, 'rating_count': rating_count,
#  'rating_1': r_1, 'rating_2': r_2, 'rating_3': r_3, 'rating_4': r_4, 'rating_5': r_5,
#  'image_url': image_url, 'small_image_url': small_image_url, 'tags': tags}, requests.get(small_image_url).status_code

[3779, 1418, 699, 524, 395, 321, 258, 205, 179, 176]

## Finally, collect data

In [91]:
import re
from bs4 import BeautifulSoup
import requests

def get_data(i):

    with open(f'newer-books/{i}.xml', 'rb') as page:
        soup = BeautifulSoup(page, 'html.parser')
    # print(soup)
    book_id = int
    goodreads_book_id = int(i)
    try: books_count = int(''.join(d for d in soup.select('.row')[0].text.strip() 
                    if d.isdigit()))
    except: books_count = None
    try: isbn = soup.select('.infoBoxRowItem')[1].text.strip()
    except: isbn = None
    try: 
        isbn13 = int(isbn[isbn.find(':')+2:-1])
    except: 
        isbn13 = None
    try: 
        isbn = int(isbn[:isbn.find('\n')])
    except: 
        isbn = None
    try: authors = soup.select('.authorName')[0].text.strip()
    except: authors = None
    try: original_publication_year = int(''.join(i for i 
                                    in soup.select('.row')[1].text.strip() 
                                    if i.isdigit())[-4:])
    except: original_publication_year = None    
    try: original_title = soup.select('.infoBoxRowItem')[0].text.strip()
    except: original_title = None 
    try: title = soup.select('#bookTitle')[0].text.strip()
    except: title = None
    try: language_code = soup.select('.infoBoxRowItem')[2].text.strip()
    except: language_code = None
    try: average_rating = float(soup.find_all('span', {'itemprop': 'ratingValue'})[0].text.strip())
    except: average_rating = NOne
    try: rating_count = int(''.join(filter(str.isdigit, 
                        soup.find_all('meta', {'itemprop': 'ratingCount'})[0].text.strip())))
    except: rating_count = None
    try: image_url = soup.find_all('img', {'id': 'coverImage'})[0].get('src')
    except: image_url = None
    try: 
        t = re.search(r'_(.+)_', image_url).span()
        small_image_url = image_url[:t[0]] + '_SX98_' + image_url[t[1]:]
    except:
        small_image_url = image_url[:-3] + '_SX98_' + image_url[-4:]
    try: 
        requests.get(small_image_url).status_code == 200
    except: 
        small_image_url = image_url

    try: 
        cdata = soup.find_all(text=re.compile("CDATA"))
        [r_5, r_4, r_3, r_2, r_1] = [int(x) for x in re.findall(r'% \((\d+)\)', cdata[7])]
    except:
        [r_5, r_4, r_3, r_2, r_1] = [None, None, None, None, None]

    book_data = {'book_id': book_id, 'goodreads_book_id': goodreads_book_id,
                 'books_count': books_count, 'isbn': isbn, 'isbn13': isbn13,
                 'authors': authors, 'original_publication_year': original_publication_year,
                 'original_title': original_title, 'title': title, 'language_code': language_code,
                 'average_rating': average_rating, 'rating_count': rating_count,
                 'rating_1': r_1, 'rating_2': r_2, 'rating_3': r_3, 'rating_4': r_4, 'rating_5': r_5,
                 'image_url': image_url, 'small_image_url': small_image_url}

    # book_pd = book_pd.append(book_data, ignore_index=True)


    # tags
    try: 
        tags_id = [int(_) for _ in eval(re.search(r'tags", (.+)\)', cdata[2]).group(1))]

        tags_count = []
        for g in soup.find_all('div', {'class': 'right'}):
            try:
                tags_count.append(int(''.join(filter(str.isdigit, g.text.strip()))))
            except: 
                continue
        
        book_tags = []
        for i in range(len(tags_id)):
            d = {'goodreads_book_id': goodreads_book_id, 
                        'tag_id': tags_id[i],
                        'count': tags_count[i]}
            book_tags.append(d)
    except:
        book_tags = [{'goodreads_book_id': goodreads_book_id, 
                        'tag_id': None,
                        'count': None}]

    
    return book_data, book_tags
    
#     print(f'Added ebook id {book_id} to the datarame. ({round(time.time()-toc, 2)}s) | ({title}) ')
# print(f'Total elapsed time: {round(time.time()-tic, 2)}s')

# book_pd.to_csv('book.csv', index=False)
# book_tags.to_csv('book_tags.csv', index=False)

In [79]:
import pickle

with open('newer-books/book_id.pkl', 'rb') as handle:
    book_ids = pickle.load(handle)

book_id = []
for i in range(len(book_ids)):
    book_id.append((book_ids[i], i+10000))
books_count = int(''.join(d for d in soup.select('.row')[0].text.strip() 
                          if d.isdigit()))
# book = [dict for i in book_id]
# book_tags = [dict for i in range(len(book_id))]
len(book_id), book_id[1], len(book), len(book_tags)

(1520, ('31443394', 10001), 1520, 1520)

In [92]:
import concurrent.futures
from tqdm import tqdm
# import pandas as pd
# import time
n = 618
# with concurrent.futures.ThreadPoolExecutor() as executor:
future = map(get_data, book_ids[n:])
for i, f in enumerate(tqdm(future, total=len(book_ids[n:]))):
    book[i+n] = f[0]
    book_tags[i+n] = f[1]
    # print(i)

import csv
for i in range(len(book)):
    book[i]['book_id'] = i + 10001
keys = book[0].keys()
with open('book.csv', 'w', newline='')  as output_file:
    dict_writer = csv.DictWriter(output_file, keys)
    dict_writer.writeheader()
    dict_writer.writerows(book)

book_tags = [item for sublist in book_tags for item in sublist]
keys = book_tags[0].keys()
with open('book_tags.csv', 'w', newline='')  as output_file:
    dict_writer = csv.DictWriter(output_file, keys)
    dict_writer.writeheader()
    dict_writer.writerows(book_tags)

  0%|          | 0/903 [00:00<?, ?it/s]


NameError: name 'NOne' is not defined

In [88]:
i+n

616

problematic:

282

428

429

616

617